**Задание 0. Загрузите необходимые библиотеки**

In [ ]:
!pip install nltk spacy pymystem3

import re
import string

import nltk
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from pymystem3 import Mystem

nltk.download("stopwords")
russian_stop_words = set(stopwords.words("russian"))
english_stop_words = set(stopwords.words("english"))

russian_stemmer = SnowballStemmer("russian")
english_stemmer = SnowballStemmer("english")

**Задание 1. Загрузка данных**

Вставьте текст для обработки согласно вашему варианту

In [41]:
sample_text = """Человеческий мозг — удивительный "компьютер", хранящий около 2,5 петабайт информации (это ≈3 млн часов видео в HD-качестве)!

Процесс формирования воспоминаний включает 3 стадии: кодирование, консолидацию и извлечение.

Гиппокамп играет ключевую роль в переводе кратковременной памяти в долговременную.

"Memory consolidation происходит преимущественно во время сна, когда мозг «перезаписывает» дневные впечатления," — объясняет доктор Миронова.

В 2022 году эксперимент с использованием технологии optogenetics позволил учёным MIT "включать" и "выключать" конкретные воспоминания у мышей!

Нейромедиатор ацетилхолин критически важен для работы памяти — его дефицит связан с болезнью Альцгеймера.

Современные исследования фокусируются на разработке методов усиления памяти с помощью brain-computer interfaces.

Представьте: скоро мы сможем загружать информацию прямо в мозг, как в фильме "Матрица" (1999)!"""

**Задание 2. Нормализация текста.**

Приведите текст к нижнему регистру, удалите лишние пробоелы, переносы строк, спецсимволы, пунктуацию, обработайте цифры.

In [42]:
def normalize_text(text):
    # Нижний регистр
    text = text.lower()

    # Заменить ё на е
    text = text.replace('ё', 'е').replace('Ё', 'Е')

    # Удаление цифр, но сохранение слов с цифрами
    text = re.sub(r'\b\d+\b', ' ', text)

    # Удалить пунктуацию
    text = re.sub(r'[^а-яa-z\s]', ' ', text)

    # Сначала заменим дефисы временным маркером
    text = re.sub(r'(\w)-(\w)', r'\1HYPHEN\2', text)

    # Удаляем пунктуацию кроме временных маркеров дефисов
    text = re.sub(r'[^\w\sHYPHEN]', ' ', text)

    # Возвращаем дефисы
    text = re.sub(r'HYPHEN', '-', text)

    # Удаление избыточных пробелов (в том числе в начале и конце строк)
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Проверка
normalized_russian = normalize_text(sample_text)
print("Исходный русский текст:", sample_text)
print("Нормализованный русский текст:", normalized_russian)

Исходный русский текст: Человеческий мозг — удивительный "компьютер", хранящий около 2,5 петабайт информации (это ≈3 млн часов видео в HD-качестве)!

Процесс формирования воспоминаний включает 3 стадии: кодирование, консолидацию и извлечение.

Гиппокамп играет ключевую роль в переводе кратковременной памяти в долговременную.

"Memory consolidation происходит преимущественно во время сна, когда мозг «перезаписывает» дневные впечатления," — объясняет доктор Миронова.

В 2022 году эксперимент с использованием технологии optogenetics позволил учёным MIT "включать" и "выключать" конкретные воспоминания у мышей!

Нейромедиатор ацетилхолин критически важен для работы памяти — его дефицит связан с болезнью Альцгеймера.

Современные исследования фокусируются на разработке методов усиления памяти с помощью brain-computer interfaces.

Представьте: скоро мы сможем загружать информацию прямо в мозг, как в фильме "Матрица" (1999)!
Нормализованный русский текст: человеческий мозг удивительный компьют

**Задание 3. Токенизация**

Токенизируйте текст.

In [43]:
def tokenize_text(text):
  tokens = text.split()
  return tokens

tokenized_text = tokenize_text(normalized_russian)
print(tokenized_text)

['человеческий', 'мозг', 'удивительный', 'компьютер', 'хранящий', 'около', 'петабайт', 'информации', 'это', 'млн', 'часов', 'видео', 'в', 'hd', 'качестве', 'процесс', 'формирования', 'воспоминаний', 'включает', 'стадии', 'кодирование', 'консолидацию', 'и', 'извлечение', 'гиппокамп', 'играет', 'ключевую', 'роль', 'в', 'переводе', 'кратковременной', 'памяти', 'в', 'долговременную', 'memory', 'consolidation', 'происходит', 'преимущественно', 'во', 'время', 'сна', 'когда', 'мозг', 'перезаписывает', 'дневные', 'впечатления', 'объясняет', 'доктор', 'миронова', 'в', 'году', 'эксперимент', 'с', 'использованием', 'технологии', 'optogenetics', 'позволил', 'ученым', 'mit', 'включать', 'и', 'выключать', 'конкретные', 'воспоминания', 'у', 'мышей', 'нейромедиатор', 'ацетилхолин', 'критически', 'важен', 'для', 'работы', 'памяти', 'его', 'дефицит', 'связан', 'с', 'болезнью', 'альцгеймера', 'современные', 'исследования', 'фокусируются', 'на', 'разработке', 'методов', 'усиления', 'памяти', 'с', 'помощью

**Задание 4. Удаление стоп-слов**

Выведите 2 списка - 1. Очищенных токенов, 2. Список удаленных стоп-слов

In [44]:
cleaned_tokens = []
removed_stopwords = []
def clean_text(tokens, russian_stop_words):

    for token in tokens:
        if token in russian_stop_words:
            removed_stopwords.append(token)
        else:
            cleaned_tokens.append(token)

    return cleaned_tokens, removed_stopwords

cleaned, removed = clean_text(tokenized_text, russian_stop_words)

print('Очищенные токены:', cleaned)
print('Удалённые стоп-слова:', removed)

Очищенные токены: ['человеческий', 'мозг', 'удивительный', 'компьютер', 'хранящий', 'около', 'петабайт', 'информации', 'это', 'млн', 'часов', 'видео', 'hd', 'качестве', 'процесс', 'формирования', 'воспоминаний', 'включает', 'стадии', 'кодирование', 'консолидацию', 'извлечение', 'гиппокамп', 'играет', 'ключевую', 'роль', 'переводе', 'кратковременной', 'памяти', 'долговременную', 'memory', 'consolidation', 'происходит', 'преимущественно', 'время', 'сна', 'мозг', 'перезаписывает', 'дневные', 'впечатления', 'объясняет', 'доктор', 'миронова', 'году', 'эксперимент', 'использованием', 'технологии', 'optogenetics', 'позволил', 'ученым', 'mit', 'включать', 'выключать', 'конкретные', 'воспоминания', 'мышей', 'нейромедиатор', 'ацетилхолин', 'критически', 'важен', 'работы', 'памяти', 'дефицит', 'связан', 'болезнью', 'альцгеймера', 'современные', 'исследования', 'фокусируются', 'разработке', 'методов', 'усиления', 'памяти', 'помощью', 'brain', 'computer', 'interfaces', 'представьте', 'скоро', 'смож

**Задание 5. Лемматизация и стемминг**

Примените к токенам алгоритмы лемматизации и стемминга. Выведите 2 списка - 1. Лемматизированные токены 2. Стемматизированные токены

In [45]:
def is_cyrillic(token):
    return any('а' <= char <= 'я' or char == 'ё' for char in token)

def is_latin(token):
    return any('a' <= char <= 'z' for char in token)

def stem_words(cleaned_tokens):
    stemmed_tokens = []

    for token in cleaned_tokens:
        if token.isalpha():
            if is_cyrillic(token):
                stem = russian_stemmer.stem(token)
                stemmed_tokens.append(stem)

            elif is_latin(token):
                stem = english_stemmer.stem(token)
                stemmed_tokens.append(stem)
    return stemmed_tokens

def lemmatize_words(cleaned_tokens):
    lemmatized_tokens = []

    for token in cleaned_tokens:
        if token.isalpha():
            if is_cyrillic(token):
                lemma = mystem.lemmatize(token)[0]
                lemmatized_tokens.append(lemma)

            elif is_latin(token):
                lemma = lemmatizer_en.lemmatize(token)
                lemmatized_tokens.append(lemma)
    return lemmatized_tokens

stemmed_tokens = stem_words(cleaned_tokens)
lemmatized_tokens = lemmatize_words(cleaned_tokens)

print('Стемматизированные токены: ', stemmed_tokens)
print('Лемматизированные токены: ', lemmatized_tokens)

Стемматизированные токены:  ['человеческ', 'мозг', 'удивительн', 'компьютер', 'храня', 'окол', 'петабайт', 'информац', 'эт', 'млн', 'час', 'виде', 'hd', 'качеств', 'процесс', 'формирован', 'воспоминан', 'включа', 'стад', 'кодирован', 'консолидац', 'извлечен', 'гиппокамп', 'игра', 'ключев', 'рол', 'перевод', 'кратковремен', 'памят', 'долговремен', 'memori', 'consolid', 'происход', 'преимуществен', 'врем', 'сна', 'мозг', 'перезаписыва', 'дневн', 'впечатлен', 'объясня', 'доктор', 'миронов', 'год', 'эксперимент', 'использован', 'технолог', 'optogenet', 'позвол', 'учен', 'mit', 'включа', 'выключа', 'конкретн', 'воспоминан', 'мыш', 'нейромедиатор', 'ацетилхолин', 'критическ', 'важ', 'работ', 'памят', 'дефиц', 'связа', 'болезн', 'альцгеймер', 'современ', 'исследован', 'фокусир', 'разработк', 'метод', 'усилен', 'памят', 'помощ', 'brain', 'comput', 'interfac', 'представьт', 'скор', 'смож', 'загружа', 'информац', 'прям', 'мозг', 'фильм', 'матриц']
Лемматизированные токены:  ['человеческий', 'моз

**Задание 6. Напишите функцию для препроцессинга текста**

Объедините все шаги в одну функцию. Выведите результат с лемматизированным списком

In [46]:
def preprocess_text(text, russian_stop_words):
    # Нормализация

    # Нижний регистр
    text = text.lower()
    # Заменить ё на е
    text = text.replace('ё', 'е').replace('Ё', 'Е')
    # Удаление цифр, но сохранение слов с цифрами
    text = re.sub(r'\b\d+\b', ' ', text)
    # Удалить пунктуацию
    text = re.sub(r'[^а-яa-z\s]', ' ', text)
    # Сначала заменим дефисы временным маркером
    text = re.sub(r'(\w)-(\w)', r'\1HYPHEN\2', text)
    # Удаляем пунктуацию кроме временных маркеров дефисов
    text = re.sub(r'[^\w\sHYPHEN]', ' ', text)
    # Возвращаем дефисы
    text = re.sub(r'HYPHEN', '-', text)
    # Удаление избыточных пробелов (в том числе в начале и конце строк)
    text = re.sub(r'\s+', ' ', text).strip()

    # Токенизация
    tokens = text.split()

    # Удаление стоп-слов
    cleaned_tokens = [
        token for token in tokens
        if token not in russian_stop_words
    ]

    # Лемматизация
    lemmatized_tokens = []

    for token in cleaned_tokens:
        if token.isalpha():
            if any('а' <= char <= 'я' for char in token):
                lemma = mystem.lemmatize(token)[0]
                lemmatized_tokens.append(lemma)

            elif any('a' <= char <= 'z' for char in token):
                lemma = lemmatizer_en.lemmatize(token)
                lemmatized_tokens.append(lemma)

    return lemmatized_tokens

print('Предобработанный текст:', lemmatized_tokens)

Предобработанный текст: ['человеческий', 'мозг', 'удивительный', 'компьютер', 'хранить', 'около', 'петабайт', 'информация', 'это', 'млн', 'час', 'видео', 'hd', 'качество', 'процесс', 'формирование', 'воспоминание', 'включать', 'стадия', 'кодирование', 'консолидация', 'извлечение', 'гиппокамп', 'играть', 'ключевой', 'роль', 'перевод', 'кратковременный', 'память', 'долговременный', 'memory', 'consolidation', 'происходить', 'преимущественно', 'время', 'сон', 'мозг', 'перезаписывать', 'дневной', 'впечатление', 'объяснять', 'доктор', 'миронова', 'год', 'эксперимент', 'использование', 'технология', 'optogenetics', 'позволять', 'ученый', 'mit', 'включать', 'выключать', 'конкретный', 'воспоминание', 'мышь', 'нейромедиатор', 'ацетилхолин', 'критически', 'важный', 'работа', 'память', 'дефицит', 'связанный', 'болезнь', 'альцгеймер', 'современный', 'исследование', 'фокусироваться', 'разработка', 'метод', 'усиление', 'память', 'помощь', 'brain', 'computer', 'interface', 'представлять', 'скоро', 'см